In [1]:
import soundfile as sf
import os, librosa
import numpy as np
import torch
import torch.nn as nn
import random
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor, Wav2Vec2Model
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
import timeit
from pynvml import *
# import opensmile
from scipy import signal
import pwcca
import torch.optim as optim

In [2]:
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-base-960h")
asr_model = Wav2Vec2Model.from_pretrained("facebook/wav2vec2-base-960h").to(DEVICE)

Some weights of the model checkpoint at facebook/wav2vec2-base-960h were not used when initializing Wav2Vec2Model: ['lm_head.weight', 'lm_head.bias']
- This IS expected if you are initializing Wav2Vec2Model from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2Model from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2Model were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [3]:
# smile = opensmile.Smile(
#     feature_set=opensmile.FeatureSet.eGeMAPSv02,
#     feature_level=opensmile.FeatureLevel.LowLevelDescriptors,
# )

# smile2 = opensmile.Smile(
#     feature_set=opensmile.FeatureSet.eGeMAPSv02,
#     feature_level=opensmile.FeatureLevel.Functionals,
# )

smile3 = opensmile.Smile(
    feature_set=opensmile.FeatureSet.ComParE_2016,
    feature_level=opensmile.FeatureLevel.LowLevelDescriptors,
)

smile4 = opensmile.Smile(
    feature_set=opensmile.FeatureSet.ComParE_2016,
    feature_level=opensmile.FeatureLevel.LowLevelDescriptors_Deltas,
)

# smile5 = opensmile.Smile(
#     feature_set=opensmile.FeatureSet.ComParE_2016,
#     feature_level=opensmile.FeatureLevel.Functionals,
# )

In [3]:
# batch data preparation 

path = '/afs/inf.ed.ac.uk/user/s20/s2057508/Documents/Corpora/RAVDESS/Data'

names_train = []
names_test = []
emolabels_train = []
emolabels_test = []

# train_hct_feats = []
# test_hct_feats = []
# train_mfcc_feats = []
# test_mfcc_feats = []

train_asr_hidden0 = []
train_asr_hidden1 = []
train_asr_hidden2 = []
train_asr_hidden3 = []
train_asr_hidden4 = []
train_asr_hidden5 = []
train_asr_hidden6 = []
train_asr_hidden7 = []
train_asr_hidden8 = []
train_asr_hidden9 = []
train_asr_hidden10 = []
train_asr_hidden11 = []
train_asr_hidden12 = []
train_asr_hiddenave = []
train_asr_hiddenrm = []

test_asr_hidden0 = []
test_asr_hidden1 = []
test_asr_hidden2 = []
test_asr_hidden3 = []
test_asr_hidden4 = []
test_asr_hidden5 = []
test_asr_hidden6 = []
test_asr_hidden7 = []
test_asr_hidden8 = []
test_asr_hidden9 = []
test_asr_hidden10 = []
test_asr_hidden11 = []
test_asr_hidden12 = []
test_asr_hiddenave = []
test_asr_hiddenrm = []

n = 1
for dirname, dirs, filenames in os.walk(path):
    for filename in filenames:
        print(n)
        emotion = filename[7]
        file_name = os.path.join(dirname, filename)
        audioinput, sr = librosa.load(file_name, sr=16000)

#         feats = smile.process_signal(audioinput, sr)

        if emotion == '1':
            emotion_label = 0
        elif emotion == '2':
            emotion_label = 1
        elif emotion == '3':
            emotion_label = 2
        elif emotion == '4':
            emotion_label = 3
        elif emotion == '5':
            emotion_label = 4
        elif emotion == '6':
            emotion_label = 5
        elif emotion == '7':
            emotion_label = 6
        elif emotion == '8':
            emotion_label = 7
        
        torch.cuda.empty_cache()
                
        inputs = processor(audioinput, sampling_rate=16000, return_tensors='pt').input_values.to(DEVICE)
        hiddens = asr_model(inputs, output_hidden_states=True).hidden_states
        hidden0 = hiddens[0].detach()[0].mean(dim=0)
        hidden1 = hiddens[1].detach()[0].mean(dim=0)
        hidden2 = hiddens[2].detach()[0].mean(dim=0)
        hidden3 = hiddens[3].detach()[0].mean(dim=0)
        hidden4 = hiddens[4].detach()[0].mean(dim=0)
        hidden5 = hiddens[5].detach()[0].mean(dim=0)
        hidden6 = hiddens[6].detach()[0].mean(dim=0)
        hidden7 = hiddens[7].detach()[0].mean(dim=0)
        hidden8 = hiddens[8].detach()[0].mean(dim=0)
        hidden9 = hiddens[9].detach()[0].mean(dim=0)
        hidden10 = hiddens[10].detach()[0].mean(dim=0)
        hidden11 = hiddens[11].detach()[0].mean(dim=0)
        hidden12 = hiddens[12].detach()[0].mean(dim=0)
        
        hidden = (hidden0 + hidden1 + hidden2 + hidden3 + hidden4 + hidden5 + hidden6 + hidden7 + hidden8 + hidden9 + hidden10 + hidden11 + hidden12) / 13.0
        hidden_rm = (hidden0 + hidden1 + hidden2 + hidden3 + hidden4 + hidden5 + hidden6 + hidden7 + hidden8 + hidden9 + hidden10) / 11.0
        
#         mfcc = librosa.feature.melspectrogram(y=audio_input, sr=sr, n_fft=400, hop_length=160).T
#         mfcc = np.mean(mfcc, axis=0)
#         feats1 = smile.process_signal(audio_input, sr).values
#         feats2 = smile2.process_signal(audio_input, sr).values
#         feats3 = smile3.process_signal(audio_input, sr).values
#         feats4 = smile4.process_signal(audio_input, sr).values
#         feats5 = smile5.process_signal(audio_input, sr).values.reshape(1, -1)

#         feats1 = np.mean(feats1, axis=0)
#         feats2 = np.mean(feats2, axis=0)
#         feats3 = np.mean(feats3, axis=0)
#         feats4 = np.mean(feats4, axis=0)
#         mfcc = mfcc.reshape(1, -1)
#         hct = np.concatenate((feats3, feats4)).reshape(1, -1)

#         print(hct.shape, hidden.shape)
             
#         if dirname[-2:] not in ['2', '8', '10', '16', '18', '24']:
#         if dirname[-2:] not in ['1', '3', '9', '11', '19', '23']:
#         if dirname[-2:] not in ['4', '6', '12', '13', '20', '21']:
        if dirname[-2:] not in ['5', '7', '14', '15', '17', '22']:
            names_train.append(file_name)
            emolabels_train.append(emotion_label)
            train_asr_hidden0.append(hidden0.view(1, 768).cpu().numpy())
            train_asr_hidden1.append(hidden1.view(1, 768).cpu().numpy())
            train_asr_hidden2.append(hidden2.view(1, 768).cpu().numpy())
            train_asr_hidden3.append(hidden3.view(1, 768).cpu().numpy())
            train_asr_hidden4.append(hidden4.view(1, 768).cpu().numpy())
            train_asr_hidden5.append(hidden5.view(1, 768).cpu().numpy())
            train_asr_hidden6.append(hidden6.view(1, 768).cpu().numpy())
            train_asr_hidden7.append(hidden7.view(1, 768).cpu().numpy())
            train_asr_hidden8.append(hidden8.view(1, 768).cpu().numpy())
            train_asr_hidden9.append(hidden9.view(1, 768).cpu().numpy())
            train_asr_hidden10.append(hidden10.view(1, 768).cpu().numpy())
            train_asr_hidden11.append(hidden11.view(1, 768).cpu().numpy())
            train_asr_hidden12.append(hidden12.view(1, 768).cpu().numpy())
            train_asr_hiddenave.append(hidden.view(1, 768).cpu().numpy())            
            train_asr_hiddenrm.append(hidden_rm.view(1, 768).cpu().numpy())

#             train_hct_feats.append(np.array(hct))
#             train_mfcc_feats.append(np.array(mfcc))
                
        else:
            names_test.append(file_name)
            emolabels_test.append(emotion_label)
            test_asr_hidden0.append(hidden0.view(1, 768).cpu().numpy())
            test_asr_hidden1.append(hidden1.view(1, 768).cpu().numpy())
            test_asr_hidden2.append(hidden2.view(1, 768).cpu().numpy())
            test_asr_hidden3.append(hidden3.view(1, 768).cpu().numpy())
            test_asr_hidden4.append(hidden4.view(1, 768).cpu().numpy())
            test_asr_hidden5.append(hidden5.view(1, 768).cpu().numpy())
            test_asr_hidden6.append(hidden6.view(1, 768).cpu().numpy())
            test_asr_hidden7.append(hidden7.view(1, 768).cpu().numpy())
            test_asr_hidden8.append(hidden8.view(1, 768).cpu().numpy())
            test_asr_hidden9.append(hidden9.view(1, 768).cpu().numpy())
            test_asr_hidden10.append(hidden10.view(1, 768).cpu().numpy())
            test_asr_hidden11.append(hidden11.view(1, 768).cpu().numpy())
            test_asr_hidden12.append(hidden12.view(1, 768).cpu().numpy())
            test_asr_hiddenave.append(hidden.view(1, 768).cpu().numpy())            
            test_asr_hiddenrm.append(hidden_rm.view(1, 768).cpu().numpy())

#             test_hct_feats.append(np.array(hct))
#             test_mfcc_feats.append(np.array(mfcc))

        n+=1            
        torch.cuda.empty_cache()

emolabels_train = np.array(emolabels_train)
emolabels_test = np.array(emolabels_test)
# train_hct_feats = torch.tensor(train_hct_feats).to(DEVICE)
# test_hct_feats = torch.tensor(test_hct_feats).to(DEVICE)
# train_mfcc_feats = torch.tensor(train_mfcc_feats).to(DEVICE)
# test_mfcc_feats = torch.tensor(test_mfcc_feats).to(DEVICE)
train_asr_hidden0 = torch.tensor(train_asr_hidden0).to(DEVICE)
train_asr_hidden1 = torch.tensor(train_asr_hidden1).to(DEVICE)
train_asr_hidden2 = torch.tensor(train_asr_hidden2).to(DEVICE)
train_asr_hidden3 = torch.tensor(train_asr_hidden3).to(DEVICE)
train_asr_hidden4 = torch.tensor(train_asr_hidden4).to(DEVICE)
train_asr_hidden5 = torch.tensor(train_asr_hidden5).to(DEVICE)
train_asr_hidden6 = torch.tensor(train_asr_hidden6).to(DEVICE)
train_asr_hidden7 = torch.tensor(train_asr_hidden7).to(DEVICE)
train_asr_hidden8 = torch.tensor(train_asr_hidden8).to(DEVICE)
train_asr_hidden9 = torch.tensor(train_asr_hidden9).to(DEVICE)
train_asr_hidden10 = torch.tensor(train_asr_hidden10).to(DEVICE)
train_asr_hidden11 = torch.tensor(train_asr_hidden11).to(DEVICE)
train_asr_hidden12 = torch.tensor(train_asr_hidden12).to(DEVICE)
train_asr_hiddenave = torch.tensor(train_asr_hiddenave).to(DEVICE)
train_asr_hiddenrm = torch.tensor(train_asr_hiddenrm).to(DEVICE)

test_asr_hidden0 = torch.tensor(test_asr_hidden0).to(DEVICE)
test_asr_hidden1 = torch.tensor(test_asr_hidden1).to(DEVICE)
test_asr_hidden2 = torch.tensor(test_asr_hidden2).to(DEVICE)
test_asr_hidden3 = torch.tensor(test_asr_hidden3).to(DEVICE)
test_asr_hidden4 = torch.tensor(test_asr_hidden4).to(DEVICE)
test_asr_hidden5 = torch.tensor(test_asr_hidden5).to(DEVICE)
test_asr_hidden6 = torch.tensor(test_asr_hidden6).to(DEVICE)
test_asr_hidden7 = torch.tensor(test_asr_hidden7).to(DEVICE)
test_asr_hidden8 = torch.tensor(test_asr_hidden8).to(DEVICE)
test_asr_hidden9 = torch.tensor(test_asr_hidden9).to(DEVICE)
test_asr_hidden10 = torch.tensor(test_asr_hidden10).to(DEVICE)
test_asr_hidden11 = torch.tensor(test_asr_hidden11).to(DEVICE)
test_asr_hidden12 = torch.tensor(test_asr_hidden12).to(DEVICE)
test_asr_hiddenave = torch.tensor(test_asr_hiddenave).to(DEVICE)
test_asr_hiddenrm = torch.tensor(test_asr_hiddenrm).to(DEVICE)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


/usr/lib/python3.6/site-packages/ipykernel_launcher.py:168: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  ../torch/csrc/utils/tensor_new.cpp:201.)


In [14]:
print(emolabels_train.shape)

(1260,)


In [29]:
train_hct_feats -= np.mean(train_hct_feats, axis = 0)
train_hct_feats /= np.std(train_hct_feats, axis = 0)
test_hct_feats -= np.mean(test_hct_feats, axis = 0)
test_hct_feats /= np.std(test_hct_feats, axis = 0)

In [31]:
train_hct_feats = torch.from_numpy(train_hct_feats)
test_hct_feats = torch.from_numpy(test_hct_feats)

In [4]:
indices_train = []
for i in range(len(names_train)):
    indices_train.append(i)

In [34]:
print(len(names_test)+len(names_train))

1440


In [12]:
# MLP
class NeuralNet(nn.Module):
    def __init__(self):
        super(NeuralNet,self).__init__()
#         self.amp = nn.Linear(90, 768)
        self.lstm = nn.LSTM(input_size=768,
                            hidden_size=64,
                            num_layers=2,
                            batch_first=True,
                            dropout=0.5,
                            bidirectional=True)
#         self.amp = nn.Linear(64, 128)
        self.attn = nn.MultiheadAttention(128, 16, batch_first=True)
        self.dense1 = nn.Linear(768, 128)
        self.flat = nn.Flatten()
        self.dense = nn.Linear(128, 16)
        self.acti = nn.ReLU()
        self.out = nn.Linear(16, 8)

    def forward(self, x):
#         x = self.amp(x)
#         self.lstm.flatten_parameters()
#         x, _ = self.lstm(x)
#         x, _ = self.attn(x,x,x)
        x = self.flat(x)
        x = self.dense1(x)
        x = self.acti(x)
        x = self.dense(x)
        res = self.acti(x)
        emotion = self.out(res)
        return emotion

In [9]:
# weighted sum
class NeuralNet(nn.Module):
    def __init__(self):
        super(NeuralNet,self).__init__()
        self.W = torch.nn.Parameter(torch.ones(1, 13), requires_grad=True)
        self.lstm = nn.LSTM(input_size=768,
                            hidden_size=64,
                            num_layers=2,
                            batch_first=True,
                            dropout=0.5,
                            bidirectional=True)
#         self.amp = nn.Linear(64, 128)
        self.attn = nn.MultiheadAttention(128, 16, batch_first=True)
        self.dense1 = nn.Linear(768, 128)
        self.flat = nn.Flatten()
        self.dense = nn.Linear(128, 16)
        self.acti = nn.ReLU()
        self.out = nn.Linear(16, 8)

    def forward(self, x):
        x = torch.matmul(self.W, x)
        x = self.flat(x)
        x = torch.div(x, torch.sum(self.W, 1))
#         print(x.size(), self.W.size())
#         print(torch.sum(self.W, 2).size())
#         x = self.amp(x)
#         self.lstm.flatten_parameters()
#         x, _ = self.lstm(x)
#         x, _ = self.attn(x,x,x)
#         x = self.flat(x)
        x = self.dense1(x)
        x = self.acti(x)
        x = self.dense(x)
        res = self.acti(x)
        emotion = self.out(res)
        return emotion, self.W

In [7]:
a = torch.randn(64, 1, 13)
b = torch.randn(64, 13, 768)
# b = torch.tensor(([[1,1,1],[1,1,1]], [[2,2,2],[2,2,2]], [[3,3,3],[3,3,3]])).to(float)
# c = torch.tensor([[3, 3, 3, 3, 3]])
# d = torch.tensor([[3, 3, 3, 3, 3]])
# print(b.size())
e = torch.matmul(a, b)
print(e.size())
# print(torch.dot(a,b))

torch.Size([64, 1, 768])


In [14]:
torch.manual_seed(1)
ser_model = NeuralNet().to(DEVICE)
params = ser_model.parameters()
optimizer = torch.optim.AdamW(params, lr=2e-4, weight_decay=1e-5)
# scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.5)

func = nn.CrossEntropyLoss()

epochs = 1000
epoch = 0
batch_size = 64


# training
while epoch < epochs:
    torch.cuda.empty_cache()
#     start = timeit.default_timer()
#     print("-----epoch: ", epoch, "-----")
    predictions_train = []
    predictions_test = []

#     print('--training begins--')
    ser_model.train()
    j = 0
    while j < len(names_train)/batch_size:
        if (j+1)*batch_size > len(names_train):
            emolabels = torch.tensor(emolabels_train[j*batch_size: ]).to(DEVICE)
            indices = indices_train[j*batch_size: ]
            asr_feats0 = train_asr_hidden0[j*batch_size: ].to(DEVICE)
            asr_feats1 = train_asr_hidden1[j*batch_size: ].to(DEVICE)
            asr_feats2 = train_asr_hidden2[j*batch_size: ].to(DEVICE)
            asr_feats3 = train_asr_hidden3[j*batch_size: ].to(DEVICE)
            asr_feats4 = train_asr_hidden4[j*batch_size: ].to(DEVICE)
            asr_feats5 = train_asr_hidden5[j*batch_size: ].to(DEVICE)
            asr_feats6 = train_asr_hidden6[j*batch_size: ].to(DEVICE)
            asr_feats7 = train_asr_hidden7[j*batch_size: ].to(DEVICE)
            asr_feats8 = train_asr_hidden8[j*batch_size: ].to(DEVICE)
            asr_feats9 = train_asr_hidden9[j*batch_size: ].to(DEVICE)
            asr_feats10 = train_asr_hidden10[j*batch_size: ].to(DEVICE)
            asr_feats11 = train_asr_hidden11[j*batch_size: ].to(DEVICE)
            asr_feats12 = train_asr_hidden12[j*batch_size: ].to(DEVICE)
            asr_featsave = train_asr_hiddenave[j*batch_size: ].to(DEVICE)
            asr_featsrm = train_asr_hiddenrm[j*batch_size: ].to(DEVICE)
            
        else:
            emolabels = torch.tensor(emolabels_train[j*batch_size:(j+1)*batch_size]).to(DEVICE)
            indices = indices_train[j*batch_size:(j+1)*batch_size]
            asr_feats0 = train_asr_hidden0[j*batch_size:(j+1)*batch_size].to(DEVICE)
            asr_feats1 = train_asr_hidden1[j*batch_size:(j+1)*batch_size].to(DEVICE)
            asr_feats2 = train_asr_hidden2[j*batch_size:(j+1)*batch_size].to(DEVICE)
            asr_feats3 = train_asr_hidden3[j*batch_size:(j+1)*batch_size].to(DEVICE)
            asr_feats4 = train_asr_hidden4[j*batch_size:(j+1)*batch_size].to(DEVICE)
            asr_feats5 = train_asr_hidden5[j*batch_size:(j+1)*batch_size].to(DEVICE)
            asr_feats6 = train_asr_hidden6[j*batch_size:(j+1)*batch_size].to(DEVICE)
            asr_feats7 = train_asr_hidden7[j*batch_size:(j+1)*batch_size].to(DEVICE)
            asr_feats8 = train_asr_hidden8[j*batch_size:(j+1)*batch_size].to(DEVICE)
            asr_feats9 = train_asr_hidden9[j*batch_size:(j+1)*batch_size].to(DEVICE)
            asr_feats10 = train_asr_hidden10[j*batch_size:(j+1)*batch_size].to(DEVICE)
            asr_feats11 = train_asr_hidden11[j*batch_size:(j+1)*batch_size].to(DEVICE)
            asr_feats12 = train_asr_hidden12[j*batch_size:(j+1)*batch_size].to(DEVICE)
            asr_featsave = train_asr_hiddenave[j*batch_size:(j+1)*batch_size].to(DEVICE)
            asr_featsrm = train_asr_hiddenrm[j*batch_size:(j+1)*batch_size].to(DEVICE)
            
        asr_weighted = torch.cat((asr_feats0,asr_feats1,asr_feats2,asr_feats3,asr_feats4,asr_feats5,asr_feats6,
                                 asr_feats7,asr_feats8,asr_feats9,asr_feats10,asr_feats11,asr_feats12), 1)
            
# ser loss
#         pred, weights = ser_model(asr_weighted)
        pred = ser_model(asr_featsave)
#         pred = ser_model(hct_feats)
#         pred = ser_model(asr_feats0, hct_feats)
#         pred = ser_model(asr_feats_g1, asr_feats_g2, asr_feats_g3, hct_feats.to(torch.float))
        train_ser_loss = func(pred, torch.tensor(emolabels))
        for i in pred:
            predictions_train.append(i.detach().cpu().numpy())
        j += 1

# backprop
        optimizer.zero_grad()
        torch.nn.utils.clip_grad_norm_(params, 1.0)
        train_ser_loss.backward()
        optimizer.step()
        torch.cuda.empty_cache()
#     print('--training ends--')


# testing
# extract features
#     print('--testing begins--')
    ser_model.eval()

# ser loss
    test_asr_weighted = torch.cat((test_asr_hidden0,test_asr_hidden1,test_asr_hidden2,test_asr_hidden3,test_asr_hidden4,
                                 test_asr_hidden5,test_asr_hidden6,test_asr_hidden7,test_asr_hidden8,test_asr_hidden9,
                                 test_asr_hidden10,test_asr_hidden11,test_asr_hidden12), 1)
#     pred, weights = ser_model(test_asr_weighted)
    pred = ser_model(test_asr_hiddenave)
#     pred = ser_model(test_hct_feats.to(DEVICE))
#     pred = ser_model(test_asr_hidden0, test_hct_feats.to(DEVICE))
#     asr_feats_g1 = (test_asr_hidden1 + test_asr_hidden2 + test_asr_hidden3 + test_asr_hidden4) / 4.0
#     asr_feats_g2 = (test_asr_hidden5 + test_asr_hidden6 + test_asr_hidden7 + test_asr_hidden8) / 4.0
#     asr_feats_g3 = (test_asr_hidden9 + test_asr_hidden10 + test_asr_hidden11 + test_asr_hidden12) / 4.0
#     pred = ser_model(asr_feats_g1, asr_feats_g2, asr_feats_g3, test_hct_feats.to(torch.float))
    test_ser_loss = func(pred, torch.tensor(emolabels_test).to(DEVICE))
    for i in pred:
        predictions_test.append(i.detach().cpu().numpy())
    torch.cuda.empty_cache()
#     k += 1
        
# compute performance for each epoch
    predictions_train = np.array(predictions_train)
    predictions_test = np.array(predictions_test)
    emolabels_train = np.array(emolabels_train)
    emolabels_test = np.array(emolabels_test)
    predictions_train = [np.argmax(p) for p in predictions_train]
    acc_train = accuracy_score(emolabels_train, predictions_train)
    predictions_test = [np.argmax(p) for p in predictions_test]
    acc_test = accuracy_score(emolabels_test, predictions_test)
    
    print('Epoch:', epoch, '|train_ser_loss: %.4f' %train_ser_loss, '|train_accuracy: %.4f' %acc_train, '|test_ser_loss: %.4f' %test_ser_loss, '|test_accuracy: %.4f' %acc_test)
#     print('test: \n', confusion_matrix(emolabels_test, predictions_test))
#     print(classification_report(emolabels_test, predictions_test))
#     print(torch.div(weights.detach().cpu(), torch.sum(weights.detach().cpu())))

    epoch += 1
#     scheduler.step()
#     print('---testing ends---')
    
#     stop = timeit.default_timer()
#     print('Time: ', stop - start) 

/usr/lib/python3.6/site-packages/ipykernel_launcher.py:73: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


Epoch: 0 |train_ser_loss: 2.0646 |train_accuracy: 0.1333 |test_ser_loss: 2.0692 |test_accuracy: 0.1333
Epoch: 1 |train_ser_loss: 2.0577 |train_accuracy: 0.1333 |test_ser_loss: 2.0620 |test_accuracy: 0.1333
Epoch: 2 |train_ser_loss: 2.0501 |train_accuracy: 0.1617 |test_ser_loss: 2.0543 |test_accuracy: 0.2333
Epoch: 3 |train_ser_loss: 2.0414 |train_accuracy: 0.2592 |test_ser_loss: 2.0459 |test_accuracy: 0.2917
Epoch: 4 |train_ser_loss: 2.0312 |train_accuracy: 0.3433 |test_ser_loss: 2.0364 |test_accuracy: 0.3500
Epoch: 5 |train_ser_loss: 2.0184 |train_accuracy: 0.3717 |test_ser_loss: 2.0251 |test_accuracy: 0.3458
Epoch: 6 |train_ser_loss: 2.0027 |train_accuracy: 0.3800 |test_ser_loss: 2.0112 |test_accuracy: 0.3708
Epoch: 7 |train_ser_loss: 1.9835 |train_accuracy: 0.3775 |test_ser_loss: 1.9946 |test_accuracy: 0.3750
Epoch: 8 |train_ser_loss: 1.9613 |train_accuracy: 0.3700 |test_ser_loss: 1.9757 |test_accuracy: 0.3667
Epoch: 9 |train_ser_loss: 1.9364 |train_accuracy: 0.3667 |test_ser_loss: 

Epoch: 81 |train_ser_loss: 1.0667 |train_accuracy: 0.5817 |test_ser_loss: 1.3106 |test_accuracy: 0.5250
Epoch: 82 |train_ser_loss: 1.0606 |train_accuracy: 0.5842 |test_ser_loss: 1.3071 |test_accuracy: 0.5250
Epoch: 83 |train_ser_loss: 1.0544 |train_accuracy: 0.5858 |test_ser_loss: 1.3036 |test_accuracy: 0.5292
Epoch: 84 |train_ser_loss: 1.0484 |train_accuracy: 0.5883 |test_ser_loss: 1.3003 |test_accuracy: 0.5292
Epoch: 85 |train_ser_loss: 1.0427 |train_accuracy: 0.5917 |test_ser_loss: 1.2970 |test_accuracy: 0.5292
Epoch: 86 |train_ser_loss: 1.0366 |train_accuracy: 0.5933 |test_ser_loss: 1.2935 |test_accuracy: 0.5333
Epoch: 87 |train_ser_loss: 1.0307 |train_accuracy: 0.5967 |test_ser_loss: 1.2902 |test_accuracy: 0.5333
Epoch: 88 |train_ser_loss: 1.0253 |train_accuracy: 0.5967 |test_ser_loss: 1.2871 |test_accuracy: 0.5333
Epoch: 89 |train_ser_loss: 1.0195 |train_accuracy: 0.5975 |test_ser_loss: 1.2839 |test_accuracy: 0.5375
Epoch: 90 |train_ser_loss: 1.0140 |train_accuracy: 0.5983 |test_

Epoch: 160 |train_ser_loss: 0.7246 |train_accuracy: 0.6925 |test_ser_loss: 1.1323 |test_accuracy: 0.5958
Epoch: 161 |train_ser_loss: 0.7214 |train_accuracy: 0.6925 |test_ser_loss: 1.1307 |test_accuracy: 0.6000
Epoch: 162 |train_ser_loss: 0.7183 |train_accuracy: 0.6917 |test_ser_loss: 1.1292 |test_accuracy: 0.6000
Epoch: 163 |train_ser_loss: 0.7151 |train_accuracy: 0.6925 |test_ser_loss: 1.1275 |test_accuracy: 0.6000
Epoch: 164 |train_ser_loss: 0.7120 |train_accuracy: 0.6925 |test_ser_loss: 1.1262 |test_accuracy: 0.6000
Epoch: 165 |train_ser_loss: 0.7088 |train_accuracy: 0.6925 |test_ser_loss: 1.1246 |test_accuracy: 0.6042
Epoch: 166 |train_ser_loss: 0.7058 |train_accuracy: 0.6925 |test_ser_loss: 1.1231 |test_accuracy: 0.6042
Epoch: 167 |train_ser_loss: 0.7027 |train_accuracy: 0.6942 |test_ser_loss: 1.1216 |test_accuracy: 0.6042
Epoch: 168 |train_ser_loss: 0.6996 |train_accuracy: 0.6950 |test_ser_loss: 1.1201 |test_accuracy: 0.6042
Epoch: 169 |train_ser_loss: 0.6966 |train_accuracy: 0.6

Epoch: 241 |train_ser_loss: 0.5203 |train_accuracy: 0.7433 |test_ser_loss: 1.0402 |test_accuracy: 0.6500
Epoch: 242 |train_ser_loss: 0.5183 |train_accuracy: 0.7442 |test_ser_loss: 1.0394 |test_accuracy: 0.6542
Epoch: 243 |train_ser_loss: 0.5165 |train_accuracy: 0.7442 |test_ser_loss: 1.0387 |test_accuracy: 0.6542
Epoch: 244 |train_ser_loss: 0.5145 |train_accuracy: 0.7442 |test_ser_loss: 1.0379 |test_accuracy: 0.6542
Epoch: 245 |train_ser_loss: 0.5127 |train_accuracy: 0.7458 |test_ser_loss: 1.0372 |test_accuracy: 0.6542
Epoch: 246 |train_ser_loss: 0.5108 |train_accuracy: 0.7492 |test_ser_loss: 1.0365 |test_accuracy: 0.6542
Epoch: 247 |train_ser_loss: 0.5090 |train_accuracy: 0.7492 |test_ser_loss: 1.0357 |test_accuracy: 0.6583
Epoch: 248 |train_ser_loss: 0.5073 |train_accuracy: 0.7508 |test_ser_loss: 1.0350 |test_accuracy: 0.6583
Epoch: 249 |train_ser_loss: 0.5053 |train_accuracy: 0.7517 |test_ser_loss: 1.0343 |test_accuracy: 0.6625
Epoch: 250 |train_ser_loss: 0.5036 |train_accuracy: 0.7

...

Epoch: 403 |train_ser_loss: 0.3161 |train_accuracy: 0.8242 |test_ser_loss: 0.9849 |test_accuracy: 0.6542
Epoch: 404 |train_ser_loss: 0.3152 |train_accuracy: 0.8250 |test_ser_loss: 0.9849 |test_accuracy: 0.6542
Epoch: 405 |train_ser_loss: 0.3147 |train_accuracy: 0.8250 |test_ser_loss: 0.9846 |test_accuracy: 0.6542
Epoch: 406 |train_ser_loss: 0.3136 |train_accuracy: 0.8250 |test_ser_loss: 0.9847 |test_accuracy: 0.6542
Epoch: 407 |train_ser_loss: 0.3131 |train_accuracy: 0.8250 |test_ser_loss: 0.9845 |test_accuracy: 0.6542
Epoch: 408 |train_ser_loss: 0.3121 |train_accuracy: 0.8250 |test_ser_loss: 0.9843 |test_accuracy: 0.6542
Epoch: 409 |train_ser_loss: 0.3112 |train_accuracy: 0.8250 |test_ser_loss: 0.9843 |test_accuracy: 0.6542
Epoch: 410 |train_ser_loss: 0.3104 |train_accuracy: 0.8250 |test_ser_loss: 0.9843 |test_accuracy: 0.6542
Epoch: 411 |train_ser_loss: 0.3096 |train_accuracy: 0.8258 |test_ser_loss: 0.9840 |test_accuracy: 0.6542
Epoch: 412 |train_ser_loss: 0.3089 |train_accuracy: 0.8

Epoch: 484 |train_ser_loss: 0.2590 |train_accuracy: 0.8542 |test_ser_loss: 0.9841 |test_accuracy: 0.6583
Epoch: 485 |train_ser_loss: 0.2586 |train_accuracy: 0.8558 |test_ser_loss: 0.9845 |test_accuracy: 0.6583
Epoch: 486 |train_ser_loss: 0.2578 |train_accuracy: 0.8558 |test_ser_loss: 0.9843 |test_accuracy: 0.6583
Epoch: 487 |train_ser_loss: 0.2574 |train_accuracy: 0.8558 |test_ser_loss: 0.9846 |test_accuracy: 0.6625
Epoch: 488 |train_ser_loss: 0.2565 |train_accuracy: 0.8558 |test_ser_loss: 0.9848 |test_accuracy: 0.6583
Epoch: 489 |train_ser_loss: 0.2563 |train_accuracy: 0.8567 |test_ser_loss: 0.9846 |test_accuracy: 0.6625
Epoch: 490 |train_ser_loss: 0.2553 |train_accuracy: 0.8567 |test_ser_loss: 0.9849 |test_accuracy: 0.6625
Epoch: 491 |train_ser_loss: 0.2548 |train_accuracy: 0.8575 |test_ser_loss: 0.9852 |test_accuracy: 0.6583
Epoch: 492 |train_ser_loss: 0.2541 |train_accuracy: 0.8575 |test_ser_loss: 0.9851 |test_accuracy: 0.6583
Epoch: 493 |train_ser_loss: 0.2536 |train_accuracy: 0.8

Epoch: 565 |train_ser_loss: 0.2155 |train_accuracy: 0.8825 |test_ser_loss: 0.9975 |test_accuracy: 0.6667
Epoch: 566 |train_ser_loss: 0.2146 |train_accuracy: 0.8817 |test_ser_loss: 0.9978 |test_accuracy: 0.6625
Epoch: 567 |train_ser_loss: 0.2144 |train_accuracy: 0.8808 |test_ser_loss: 0.9982 |test_accuracy: 0.6667
Epoch: 568 |train_ser_loss: 0.2141 |train_accuracy: 0.8825 |test_ser_loss: 0.9977 |test_accuracy: 0.6667
Epoch: 569 |train_ser_loss: 0.2134 |train_accuracy: 0.8825 |test_ser_loss: 0.9983 |test_accuracy: 0.6667
Epoch: 570 |train_ser_loss: 0.2127 |train_accuracy: 0.8825 |test_ser_loss: 0.9989 |test_accuracy: 0.6625
Epoch: 571 |train_ser_loss: 0.2126 |train_accuracy: 0.8825 |test_ser_loss: 0.9988 |test_accuracy: 0.6667
Epoch: 572 |train_ser_loss: 0.2120 |train_accuracy: 0.8825 |test_ser_loss: 0.9991 |test_accuracy: 0.6667
Epoch: 573 |train_ser_loss: 0.2114 |train_accuracy: 0.8825 |test_ser_loss: 0.9995 |test_accuracy: 0.6625
Epoch: 574 |train_ser_loss: 0.2111 |train_accuracy: 0.8

Epoch: 646 |train_ser_loss: 0.1776 |train_accuracy: 0.9067 |test_ser_loss: 1.0231 |test_accuracy: 0.6708
Epoch: 647 |train_ser_loss: 0.1773 |train_accuracy: 0.9067 |test_ser_loss: 1.0233 |test_accuracy: 0.6708
Epoch: 648 |train_ser_loss: 0.1771 |train_accuracy: 0.9067 |test_ser_loss: 1.0244 |test_accuracy: 0.6708
Epoch: 649 |train_ser_loss: 0.1765 |train_accuracy: 0.9067 |test_ser_loss: 1.0246 |test_accuracy: 0.6708
Epoch: 650 |train_ser_loss: 0.1764 |train_accuracy: 0.9075 |test_ser_loss: 1.0242 |test_accuracy: 0.6708
Epoch: 651 |train_ser_loss: 0.1760 |train_accuracy: 0.9075 |test_ser_loss: 1.0245 |test_accuracy: 0.6708
Epoch: 652 |train_ser_loss: 0.1754 |train_accuracy: 0.9075 |test_ser_loss: 1.0247 |test_accuracy: 0.6708
Epoch: 653 |train_ser_loss: 0.1752 |train_accuracy: 0.9075 |test_ser_loss: 1.0255 |test_accuracy: 0.6708
Epoch: 654 |train_ser_loss: 0.1748 |train_accuracy: 0.9075 |test_ser_loss: 1.0260 |test_accuracy: 0.6708
Epoch: 655 |train_ser_loss: 0.1736 |train_accuracy: 0.9

KeyboardInterrupt: 

In [61]:
print(torch.div(weights.detach().cpu(), torch.sum(weights.detach().cpu())))

tensor([[0.0343, 0.0257, 0.0541, 0.1155, 0.1437, 0.1687, 0.1352, 0.1152, 0.0683,
         0.0304, 0.0299, 0.0429, 0.0361]])


In [62]:
print((0.7222+0.6583+0.6333+0.7250)/4.0)

0.6847


In [116]:
#late fusion

torch.manual_seed(1)
ser_model1 = NeuralNet1().to(DEVICE)
ser_model2 = NeuralNet2().to(DEVICE)
params1 = ser_model1.parameters()
params2 = ser_model2.parameters()
optimizer1 = torch.optim.Adam(params1, lr=5e-4, weight_decay=5e-5)
optimizer2 = torch.optim.Adam(params2, lr=1e-4, weight_decay=1e-5)
func = nn.CrossEntropyLoss()

epochs = 100
epoch = 0
batch_size = 64

# training
while epoch < epochs:
    torch.cuda.empty_cache()
#     start = timeit.default_timer()
    print("-----epoch: ", epoch, "-----")
    predictions_train = []
    predictions1_train = []
    predictions2_train = []
    predictions_test = []
    predictions1_test = []
    predictions2_test = []

#     print('--training begins--')
    ser_model.train()
    j = 0
    while j < len(names_train)/batch_size:
        if (j+1)*batch_size > len(names_train):
            emolabels = torch.tensor(emolabels_train[j*batch_size: ]).to(DEVICE)
            indices = indices_train[j*batch_size: ]
            asr_feats0 = train_asr_hidden0[j*batch_size: ].to(DEVICE)
#             asr_feats1 = train_asr_hidden1[j*batch_size: ].to(DEVICE)
#             asr_feats2 = train_asr_hidden2[j*batch_size: ].to(DEVICE)
#             asr_feats3 = train_asr_hidden3[j*batch_size: ].to(DEVICE)
#             asr_feats4 = train_asr_hidden4[j*batch_size: ].to(DEVICE)
#             asr_feats5 = train_asr_hidden5[j*batch_size: ].to(DEVICE)
#             asr_feats6 = train_asr_hidden6[j*batch_size: ].to(DEVICE)
#             asr_feats7 = train_asr_hidden7[j*batch_size: ].to(DEVICE)
#             asr_feats8 = train_asr_hidden8[j*batch_size: ].to(DEVICE)
#             asr_feats9 = train_asr_hidden9[j*batch_size: ].to(DEVICE)
#             asr_feats10 = train_asr_hidden10[j*batch_size: ].to(DEVICE)
#             asr_feats11 = train_asr_hidden11[j*batch_size: ].to(DEVICE)
#             asr_feats12 = train_asr_hidden12[j*batch_size: ].to(DEVICE)
            hct_feats = train_hct_feats[j*batch_size: ].to(DEVICE)
            
        else:
            emolabels = torch.tensor(emolabels_train[j*batch_size:(j+1)*batch_size]).to(DEVICE)
            indices = indices_train[j*batch_size:(j+1)*batch_size]
            asr_feats0 = train_asr_hidden0[j*batch_size:(j+1)*batch_size].to(DEVICE)
#             asr_feats1 = train_asr_hidden1[j*batch_size:(j+1)*batch_size].to(DEVICE)
#             asr_feats2 = train_asr_hidden2[j*batch_size:(j+1)*batch_size].to(DEVICE)
#             asr_feats3 = train_asr_hidden3[j*batch_size:(j+1)*batch_size].to(DEVICE)
#             asr_feats4 = train_asr_hidden4[j*batch_size:(j+1)*batch_size].to(DEVICE)
#             asr_feats5 = train_asr_hidden5[j*batch_size:(j+1)*batch_size].to(DEVICE)
#             asr_feats6 = train_asr_hidden6[j*batch_size:(j+1)*batch_size].to(DEVICE)
#             asr_feats7 = train_asr_hidden7[j*batch_size:(j+1)*batch_size].to(DEVICE)
#             asr_feats8 = train_asr_hidden8[j*batch_size:(j+1)*batch_size].to(DEVICE)
#             asr_feats9 = train_asr_hidden9[j*batch_size:(j+1)*batch_size].to(DEVICE)
#             asr_feats10 = train_asr_hidden10[j*batch_size:(j+1)*batch_size].to(DEVICE)
#             asr_feats11 = train_asr_hidden11[j*batch_size:(j+1)*batch_size].to(DEVICE)
#             asr_feats12 = train_asr_hidden12[j*batch_size:(j+1)*batch_size].to(DEVICE)
            hct_feats = train_hct_feats[j*batch_size:(j+1)*batch_size].to(DEVICE)
            
#         print(asr_feats_g1.shape, hct_utter.shape)
            
#         inputs = (asr_feats1 + asr_feats2 + asr_feats3 + asr_feats4 + asr_feats5 + asr_feats6 + asr_feats7 + asr_feats8 + asr_feats9 + asr_feats10 + asr_feats11 + asr_feats12) / 12.0

# ser loss
        pred1 = ser_model1(asr_feats0)
        train_ser_loss1 = func(pred1, torch.tensor(emolabels))
        for i in pred1:
            predictions1_train.append(i.detach().cpu().numpy())
        pred2 = ser_model2(hct_feats)
        train_ser_loss2 = func(pred2, torch.tensor(emolabels))
        for i in pred2:
            predictions2_train.append(i.detach().cpu().numpy())
        j += 1       

# backprop
        optimizer1.zero_grad()
        torch.nn.utils.clip_grad_norm_(params1, 1.0)
        train_ser_loss1.backward()        
        optimizer1.step()
        if epoch < 30:
            optimizer2.zero_grad()
            torch.nn.utils.clip_grad_norm_(params2, 1.0)
            train_ser_loss2.backward()        
            optimizer2.step()
        torch.cuda.empty_cache()
#     print('--training ends--')


# testing
# extract features
#     print('--testing begins--')
    ser_model.eval()

# ser loss
#     inputs_test = (test_asr_hidden1 + test_asr_hidden2 + test_asr_hidden3 + test_asr_hidden4+ test_asr_hidden5 + test_asr_hidden6 + test_asr_hidden7 + test_asr_hidden8 + test_asr_hidden9 + test_asr_hidden10 + test_asr_hidden11 + test_asr_hidden12) / 12.0
    pred1 = ser_model1(test_asr_hidden0)
    pred2 = ser_model2(test_hct_feats.to(DEVICE))
    test_ser_loss1 = func(pred1, torch.tensor(emolabels_test).to(DEVICE))
    test_ser_loss2 = func(pred2, torch.tensor(emolabels_test).to(DEVICE))
    for i in pred1:
        predictions1_test.append(i.detach().cpu().numpy())
    for i in pred2:
        predictions2_test.append(i.detach().cpu().numpy())
    torch.cuda.empty_cache()
#     k += 1
        
# compute performance for each epoch
    predictions1_train = np.array(predictions1_train)
    predictions2_train = np.array(predictions2_train)
    predictions1_test = np.array(predictions1_test)
    predictions2_test = np.array(predictions2_test)
    emolabels_train = np.array(emolabels_train)
    emolabels_test = np.array(emolabels_test)
    for i in range(len(predictions1_train)):
        if np.argmax(predictions1_train[i] == np.argmax(predictions2_train[i])):
            predictions_train.append([np.argmax(predictions1_train[i])])
        else:
            if max(predictions1_train[i]) > max(predictions2_train[i]):
                predictions_train.append([np.argmax(predictions1_train[i])])
            else:
                predictions_train.append([np.argmax(predictions2_train[i])])
#             predictions_train.append([np.argmax(predictions1_train[i] + 2*predictions2_train[i])])

    acc_train = accuracy_score(emolabels_train, predictions_train)
    for i in range(len(predictions1_test)):
        if np.argmax(predictions1_test[i] == np.argmax(predictions2_test[i])):
            predictions_test.append([np.argmax(predictions1_test[i])])
        else:
            if max(predictions1_test[i]) > max(predictions2_test[i]):
                predictions_test.append([np.argmax(predictions1_test[i])])
            else:
                predictions_test.append([np.argmax(predictions2_test[i])])
#             predictions_test.append([np.argmax(predictions1_test[i] + 2*predictions2_test[i])])
    acc_test = accuracy_score(emolabels_test, predictions_test)
 
    print('Epoch:', epoch, '|train_ser_loss: %.4f' %train_ser_loss, '|train_accuracy: %.4f' %acc_train, '|test_ser_loss1: %.4f' %test_ser_loss1, '|test_ser_loss2: %.4f' %test_ser_loss2, '|test_accuracy: %.4f' %acc_test)
#     print('test: \n', confusion_matrix(emolabels_test, predictions_test))
#     print(classification_report(emolabels_test, predictions_test))

    epoch += 1
#     scheduler.step()
#     print('---testing ends---')
    
#     stop = timeit.default_timer()
#     print('Time: ', stop - start) 

-----epoch:  0 -----


/usr/lib/python3.6/site-packages/ipykernel_launcher.py:74: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
/usr/lib/python3.6/site-packages/ipykernel_launcher.py:78: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


Epoch: 0 |train_ser_loss: 0.1982 |train_accuracy: 0.2719 |test_ser_loss1: 1.3281 |test_ser_loss2: 1.3821 |test_accuracy: 0.3747
-----epoch:  1 -----
Epoch: 1 |train_ser_loss: 0.1982 |train_accuracy: 0.4094 |test_ser_loss1: 1.1999 |test_ser_loss2: 1.3470 |test_accuracy: 0.4457
-----epoch:  2 -----
Epoch: 2 |train_ser_loss: 0.1982 |train_accuracy: 0.4874 |test_ser_loss1: 1.0775 |test_ser_loss2: 1.2607 |test_accuracy: 0.4906
-----epoch:  3 -----
Epoch: 3 |train_ser_loss: 0.1982 |train_accuracy: 0.5084 |test_ser_loss1: 1.0410 |test_ser_loss2: 1.1834 |test_accuracy: 0.4849
-----epoch:  4 -----
Epoch: 4 |train_ser_loss: 0.1982 |train_accuracy: 0.5194 |test_ser_loss1: 1.0136 |test_ser_loss2: 1.1579 |test_accuracy: 0.5061
-----epoch:  5 -----
Epoch: 5 |train_ser_loss: 0.1982 |train_accuracy: 0.5351 |test_ser_loss1: 1.0023 |test_ser_loss2: 1.1417 |test_accuracy: 0.5143
-----epoch:  6 -----
Epoch: 6 |train_ser_loss: 0.1982 |train_accuracy: 0.5548 |test_ser_loss1: 0.9856 |test_ser_loss2: 1.1264 |

Epoch: 55 |train_ser_loss: 0.1982 |train_accuracy: 0.7377 |test_ser_loss1: 0.9659 |test_ser_loss2: 1.1302 |test_accuracy: 0.6090
-----epoch:  56 -----
Epoch: 56 |train_ser_loss: 0.1982 |train_accuracy: 0.7410 |test_ser_loss1: 0.9948 |test_ser_loss2: 1.1383 |test_accuracy: 0.6090
-----epoch:  57 -----
Epoch: 57 |train_ser_loss: 0.1982 |train_accuracy: 0.7356 |test_ser_loss1: 0.9411 |test_ser_loss2: 1.1371 |test_accuracy: 0.6327
-----epoch:  58 -----
Epoch: 58 |train_ser_loss: 0.1982 |train_accuracy: 0.7377 |test_ser_loss1: 0.9742 |test_ser_loss2: 1.1279 |test_accuracy: 0.6098
-----epoch:  59 -----
Epoch: 59 |train_ser_loss: 0.1982 |train_accuracy: 0.7496 |test_ser_loss1: 0.9788 |test_ser_loss2: 1.1326 |test_accuracy: 0.6261
-----epoch:  60 -----
Epoch: 60 |train_ser_loss: 0.1982 |train_accuracy: 0.7433 |test_ser_loss1: 0.9681 |test_ser_loss2: 1.1394 |test_accuracy: 0.6147
-----epoch:  61 -----
Epoch: 61 |train_ser_loss: 0.1982 |train_accuracy: 0.7450 |test_ser_loss1: 0.9698 |test_ser_lo

In [45]:
print(len(predictions1_test))

1150


In [ ]:
--cross-domain--

session1
fusion: 0.6586

In [ ]:
--concate--

session 1
layer 6: 0.6589
hct: 0.4856

+hct: 0.6499
+layer 0: 0.6598
+layer 1: 0.6598
+layer 2: 0.6580
+layer 3: 0.6468
+layer 4: 0.6524
+layer 5: 0.6552
+layer 6: 0.6617
+layer 7: 0.6477
+layer 8: 0.6747
+layer 9: 0.6794
+layer 10: 0.6682
+layer 11: 0.6645
+layer 12: 0.6542
    

session 2
layer 6: 0.6660
hct: 0.5132

+hct: 0.6701
+layer 0: 0.6944
+layer 1: 0.6876
+layer 2: 0.6680
+layer 3: 0.6709
+layer 4: 0.6846
+layer 5: 0.6876
+layer 6: 0.6787
+layer 7: 0.6758
+layer 8: 0.6827
+layer 9: 0.6778
+layer 10: 0.6748
+layer 11: 0.6709
+layer 12: 0.6709
    
    
session 3
layer 6: 0.6322
hct: 0.4539

+hct: 0.6426
+layer 0: 0.6609
+layer 1: 0.6530
+layer 2: 0.6530
+layer 3: 0.6496
+layer 4: 0.6391
+layer 5: 0.6452
+layer 6: 0.6617
+layer 7: 0.6626
+layer 8: 0.6678
+layer 9: 0.6643
+layer 10: 0.6609
+layer 11: 0.6470
+layer 12: 0.6574 
    

session 4
layer 6: 0.6335
hct: 0.4552

+hct: 0.6442
+layer 0: 0.6481
+layer 1: 0.6511
+layer 2: 0.6559
+layer 3: 0.6608
+layer 4: 0.6472
+layer 5: 0.6511
+layer 6: 0.6442
+layer 7: 0.6520
+layer 8: 0.6550
+layer 9: 0.6618
+layer 10: 0.6481
+layer 11: 0.6472
+layer 12: 0.6442


session 5
layer 6: 0.6278
hct: 0.4914

+hct: 0.6608
+layer 0: 0.6645
+layer 1: 0.6727
+layer 2: 0.6759
+layer 3: 0.6890
+layer 4: 0.6922
+layer 5: 0.6629
+layer 6: 0.6522
+layer 7: 0.6555
+layer 8: 0.6506
+layer 9: 0.6637
+layer 10: 0.6588
+layer 11: 0.6498
+layer 12: 0.6490 general phonetic information is captured by the last layer?

In [ ]:
--co-attention--

session 1
    
+hct: 0.6524
+layer 0: 0.6449
+layer 1: 0.6505
+layer 2: 0.6514
+layer 3: 0.6542
+layer 4: 0.6421
+layer 5: 0.6701
+layer 6: 0.6580
+layer 7: 0.6514
+layer 8: 0.6570
+layer 9: 0.6775
+layer 10: 0.6719
+layer 11: 0.6505
+layer 12: 0.6552
    
    
session 2

+hct: 0.6954
+layer 0: 0.6601
+layer 1: 0.6543
+layer 2: 0.6562
+layer 3: 0.6699
+layer 4: 0.6738
+layer 5: 0.6719
+layer 6: 0.6719
+layer 7: 0.6660
+layer 8: 0.6925
+layer 9: 0.6670
+layer 10: 0.6748
+layer 11: 0.6621
+layer 12: 0.6601
    
    
session 3

+hct: 0.6278
+layer 0: 0.6365
+layer 1: 0.6374
+layer 2: 0.6557
+layer 3: 0.6487
+layer 4: 0.6661
+layer 5: 0.6591
+layer 6: 0.6513
+layer 7: 0.6591
+layer 8: 0.6626
+layer 9: 0.6696
+layer 10: 0.6470
+layer 11: 0.6435
+layer 12: 0.6339
    
    
session 4

+hct: 0.6355
+layer 0: 0.6365
+layer 1: 0.6462
+layer 2: 0.6520
+layer 3: 0.6637
+layer 4: 0.6647
+layer 5: 0.6326
+layer 6: 0.6316
+layer 7: 0.6540
+layer 8: 0.6452
+layer 9: 0.6365
+layer 10: 0.6345
+layer 11: 0.6384
+layer 12: 0.6238


session 5

+hct: 0.6400
+layer 0: 0.6531
+layer 1: 0.6661
+layer 2: 0.6571
+layer 3: 0.6784
+layer 4: 0.6857
+layer 5: 0.6531
+layer 6: 0.6506
+layer 7: 0.6539
+layer 8: 0.6596
+layer 9: 0.6506
+layer 10: 0.6441
+layer 11: 0.6384
+layer 12: 0.6335

In [ ]:
--average--

session 1: 0.6365
session 2: 0.6533
session 3: 0.6183
session 4: 0.6530
session 5: 0.6163